In [1]:
import yfinance as yf
from datetime import date, timedelta
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType, DateType, TimestampType, FloatType
from pyspark.sql.functions import udf, col, lit

#### call lib

In [2]:
%run ./work/lib.ipynb

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-86f0bd11-419b-43d7-9ee6-92eb0b2e5148;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 375ms :: artifacts dl 8ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runt

##### base date

In [3]:
load_years = 15
date_today = date.today()
base_date = date_today - timedelta(days= 30 * 12 * load_years)
base_date = date(base_date.year, 1, 1)

##### stock list

In [4]:
stocks_df = \
    spark \
        .read \
        .format('csv') \
        .options(header='True', delimiter=';', encoding='iso-8859-1') \
        .csv('{}/extract/stock_list.csv'.format(LAKE_HOME)) \
        .select('TICKER')

stock_list = \
    stocks_df \
        .rdd \
        .map(lambda x: x[0]) \
        .collect()

In [5]:
def get_stock_price(tickers: list, start: str):

    df = None

    schema = \
    StructType(
    [
        StructField('date', DateType(), True),
        StructField('open', FloatType(), True),
        StructField('high', FloatType(), True),
        StructField('low', FloatType(), True),
        StructField('close', FloatType(), True),
        StructField('adj_close', FloatType(), True),
        StructField('volume', IntegerType(), True)
    ]
    )

    for ticker in tickers:

        ticker_request = ticker + '.SA'

        data = yf.download(tickers=ticker_request, start=start)
        data.reset_index(inplace=True)

        if df == None:
            df = spark.createDataFrame(data, schema)
            df = \
                df.withColumn('ticker', lit(ticker))
        
        else:
            ticker_df = spark.createDataFrame(data, schema)
            ticker_df = \
                ticker_df.withColumn('ticker', lit(ticker))
            
            df = df.unionAll(ticker_df)

    return df

#### fact table

In [6]:
fact_price = get_stock_price(stock_list, base_date)

fact_price = \
    fact_price \
        .withColumn('ticker', name_adjust(col('ticker')))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

##### Write to DW

In [10]:
df = fact_price

write_to_dw(df, 'fact_price')